In [ ]:
import numpy as np
import pandas as pd

# from astropy.cosmology import Planck18 as cosmo
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Tcmb0=2.725)
import astropy.units as u

from load_jpas_catalogs import load_minijpas_jnep
from my_functions import *

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
from visual_inspection import load_sdss_xmatch
sdss_xm_num, sdss_xm_tid, sdss_xm_spObjID, f_zsp, xm_zsp = load_sdss_xmatch() 
tile_id, number = load_minijpas_jnep(selection=True)[4:6]
selection = pd.read_csv('csv/selection.csv')

L_Arr = np.zeros_like(tile_id).astype(float)
L_Arr_err = np.zeros_like(tile_id).astype(float)
L_spec = np.zeros_like(tile_id).astype(float)
L_spec_err = np.zeros_like(tile_id).astype(float)
nice_lya = np.zeros_like(tile_id).astype(bool)
for ii, src in enumerate(selection['src']):
    if np.abs(z_NB(selection['nb_sel'][ii]) - selection['SDSS_zspec'][ii]) < 0.15:
        nice_lya[src] = True
    else:
        continue
    L_Arr[src] = selection['L_lya'][ii]
    L_Arr_err[src] = np.log10(
        (selection['L_lya_err'][ii] + 10**L_Arr[src])
    ) - L_Arr[src]

nice_lya_and_xm = np.zeros_like(tile_id).astype(bool)

nice_xm = np.zeros_like(xm_zsp).astype(bool)
for i in range(len(sdss_xm_num)):
    try:
        mj_src = np.where((number == sdss_xm_num[i]) & (tile_id == sdss_xm_tid[i]))[0][0]
    except:
        continue
    nice_xm[i] = nice_lya[mj_src]
    nice_lya_and_xm[mj_src] = nice_lya[mj_src]

Lya_fts = pd.read_csv('csv/Lya_fts_DR16_v2.csv')

L_lya =  np.zeros_like(f_zsp)
L_lya_err =  np.zeros_like(f_zsp)
EW_lya =  np.zeros_like(f_zsp)
EW_lya_err =  np.ones_like(f_zsp) * 99
for i, this_spObjID in enumerate(sdss_xm_spObjID):
    if this_spObjID == 0:
        continue
    # Disgregate SpObjID in mjd, tile, fiber
    spObj_binary = np.binary_repr(this_spObjID)
    plate = int(spObj_binary[::-1][50:64][::-1], 2)
    mjd = int(spObj_binary[::-1][24:38][::-1], 2) + 50000
    fiber = int(spObj_binary[::-1][38:50][::-1], 2)

    wh_in_fts = np.where((plate == Lya_fts['plate'])
                         & (mjd == Lya_fts['mjd'])
                         & (fiber == Lya_fts['fiberid']))[0]
    if len(wh_in_fts) == 0:
        continue

    F_line = np.array(Lya_fts['LyaF'])[wh_in_fts] * 1e-17
    F_line_err = np.array(Lya_fts['LyaF_err'])[wh_in_fts] * 1e-17
    z = Lya_fts['Lya_z'].to_numpy().flatten()[wh_in_fts]
    dL = cosmo.luminosity_distance(z).to(u.cm).value
    L = np.log10(F_line * 4*np.pi * dL ** 2)
    L_err = np.log10(10**L * (1 / F_line * F_line_err + 1)) - L

    L_lya[i] = L
    L_lya_err[i] = L_err
    EW_lya[i] = Lya_fts['LyaEW'][wh_in_fts]
    EW_lya_err[i] = Lya_fts['LyaEW_err'][wh_in_fts]

EW_lya_err = np.abs(EW_lya_err)

for i in range(len(sdss_xm_num)):
    try:
        mj_src = np.where((number == sdss_xm_num[i]) & (tile_id == sdss_xm_tid[i]))[0][0]
    except:
        continue
    L_spec[mj_src] = L_lya[i]
    L_spec_err[mj_src] = L_lya_err[i]

In [ ]:
print(sum(nice_lya_and_xm))

In [ ]:
from scipy.stats import binned_statistic

L_Lbin_err_plus = np.load('npy/L_nb_err_plus.npy')
L_Lbin_err_minus = np.load('npy/L_nb_err_minus.npy')
median_L = np.load('npy/L_bias.npy')
L_binning = np.load('npy/L_nb_err_binning.npy')
L_bin_c = [L_binning[i: i + 2].sum() * 0.5 for i in range(len(L_binning) - 1)]

# Apply bin err
L_binning_position = binned_statistic(10 ** L_Arr, None,
                                        'count', bins=L_binning).binnumber
L_binning_position[L_binning_position > len(
    L_binning) - 2] = len(L_binning) - 2
L_e_Arr = (L_Lbin_err_plus + L_Lbin_err_minus)[L_binning_position] * 0.5

# Correct L_Arr with the median
mask_median_L = (median_L < 10)
L_Arr_corr = L_Arr + np.interp(L_Arr, np.log10(L_bin_c)
                                [mask_median_L], median_L[mask_median_L])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.errorbar(L_spec[nice_lya_and_xm], L_Arr_corr[nice_lya_and_xm],
            yerr=L_e_Arr[nice_lya_and_xm],
            xerr=L_spec_err[nice_lya_and_xm],
            ls='', fmt='o', c='k', capsize=3)
mask = nice_lya_and_xm & (L_spec > 40) & (L_Arr > 40) & (L_spec < 50) & (L_Arr < 50)
print(np.nanmedian(L_spec[mask] - L_Arr_corr[mask]))
print(np.nanstd(L_spec[mask] - L_Arr_corr[mask]))

np.save('/home/alberto/Desktop/x.npy', L_spec[nice_lya_and_xm])
np.save('/home/alberto/Desktop/y.npy', L_Arr_corr[nice_lya_and_xm])
np.save('/home/alberto/Desktop/xerr.npy', L_spec_err[nice_lya_and_xm])
np.save('/home/alberto/Desktop/yerr.npy', L_e_Arr[nice_lya_and_xm])

xx = [-100, 100]
ax.plot(xx, xx, ls='--', c='k')

ax.set_ylabel(r'Measured  $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
              fontsize=20)
ax.set_xlabel(r'Spectroscopic $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
              fontsize=20)

ax.set_xlim(42.9, 45.6)
ax.set_ylim(42.9, 45.6)

ax.tick_params(labelsize=20, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_xticks(np.arange(43, 46, 0.5))
ax.set_yticks(np.arange(43, 46, 0.5))

fig.savefig('figures/L_lya_jpas_vs_sdss.pdf', bbox_inches='tight',
            pad_inches=0.1, facecolor='w')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))

ax.errorbar(L_spec[nice_lya_and_xm], L_Arr_corr[nice_lya_and_xm] - L_spec[nice_lya_and_xm],
            ls='', fmt='o', yerr=(L_e_Arr[nice_lya_and_xm]**2 + L_spec_err[nice_lya_and_xm]**2)**0.5,
            c='k', capsize=3)
mask = nice_lya_and_xm & (L_spec > 40) & (L_Arr > 40) & (L_spec < 50) & (L_Arr < 50)
print(np.nanmedian(L_spec[mask] - L_Arr_corr[mask]))

ax.axhline(0, ls='--', c='k')

ax.set_ylabel(r'Measured  $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
              fontsize=20)
ax.set_xlabel(r'Spectroscopic $\log_{10}(L_{\mathrm{Ly}\alpha}/\mathrm{erg\,s}^{-1})$',
              fontsize=20)

ax.set_xlim(42.9, 45.6)
ax.set_ylim(-1, 1)

ax.tick_params(labelsize=20, direction='in', which='both')
ax.yaxis.set_ticks_position('both')
ax.xaxis.set_ticks_position('both')

ax.set_xticks(np.arange(43, 46, 0.5))

plt.show()